In [170]:
import geopandas as gpd
import gdal as gd
import matplotlib.pyplot as plt
from matplotlib.pyplot import axis
import pandas as pd

## Inputs and Adjustments

In [171]:
# csv of the snow depth measurements 
depths = 'SnowDepthMeasurements_TVDW.csv'

## Import Data

In [189]:
# Read the files that Thomas Sent
df0 = gpd.read_file(depths)

# collected files from 20210211
df1 = pd.read_csv('LOG01.TXT')
df1.to_csv(r'2021_02_11_probe.csv', index = None)
df1 = gpd.read_file('2021_02_11_probe.csv')


# collect second log
df2 = pd.read_csv('LOG02.TXT')
df2.to_csv(r'2021_02_11_probe2.csv', index = None)
df2 = gpd.read_file('2021_02_11_probe2.csv')
df0

,Date,Time,Latitude,Longitude,Elevation,Depth,geometry
0,1/8/2021,1:41:00 PM,43.7396278,-116.1203842,5254,0.81,None
1,1/8/2021,1:42:00 PM,43.7396202,-116.1203613,5254,0.56,None
2,1/8/2021,1:43:00 PM,43.7396164,-116.1203461,5254,0.58,None
3,1/8/2021,1:44:00 PM,43.7396049,-116.1202927,5253,0.56,None
4,1/8/2021,1:45:00 PM,43.7396011,-116.1202698,5252,0.84,None
...,...,...,...,...,...,...,...
148,1/20/2021,2:56:15 PM,43.7356644,-116.1228409,5150,0.75,None
149,1/20/2021,2:57:03 PM,43.7357330,-116.1228714,5146,0.87,None
150,1/20/2021,2:58:50 PM,43.7356300,-116.1229095,5144,0.87,None
151,1/20/2021,2:59:47 PM,43.7354584,-116.1229935,5140,0.55,None


In [188]:

df_old = df1

# turn appropriate columsn to intergers
df_old = df_old.astype({'DistanceAvg' : 'int' , 
    'Distance1' : 'int' ,
    'Distance2' : 'int' ,
    'Distance3' : 'int' ,
    'Distance4' : 'int' ,
    'Distance5' : 'int' ,
    'altitude' : 'int' }) 
df_old = df_old[df_old['DistanceAvg'] != 0] # take out any points where the average is 0

# set the titles of the new dataframe
df_new = pd.DataFrame( columns = ['Date' , 'Time' , 'Latitude' , 'Longitude' , 'Elevation' , 'Depth' , 'geometry'])

# pull out lat and long
df_new['Latitude'] = df_old['latitude']
df_new['Longitude'] = df_old['longitude']
df_new['Elevation'] = df_old['altitude']
df_new['geometry'] = df_old['geometry']


# extract the date from the original dataframe 
df_old['startGPSTime'] = pd.to_datetime(df_old['startGPSTime']) # turn the date to a datetime format
df_new['Date'] = df_old['startGPSTime'].dt.strftime('%m/%d/%Y') # properly format date to match previous dataframe
df_new['Time'] = df_old['startGPSTime'].dt.strftime('%H:%M:%S') # properly format time to match, taking 

########################## Really think something is up here and I need to talk to thomas about this because most of the heights are greater than the probe#######################################################
# calculate the Snow Heights
dst =  df_old[['Distance1' , 'Distance2' , 'Distance3' , 'Distance4' , 'Distance5']].copy().astype('int') # create a frame of heights
probe_height = dst.iloc[0].mode().astype('int') # pull probe height from mode of measurement 0
probe_height = probe_height[0]/100 # pull out the interger from the series and convert to m

raw = dst.median(axis = 'columns').astype('int') # pull out the median from the height measurements
raw = raw/100 # convert to m
# df_new['Depth'] -= probe_height # subtract probe height from measured distance to get the height of snow

# data cleaning
df_new['Depth'] = probe_height-raw # difference between probe height and raw measurements
df_new['Depth'] = raw - probe_height # difference between raw measurements and  probe height 
df_new = df_new[df_new['Depth'] >= 0] # take out any points where the average is 0
df_new = df_new[df_new['Depth'] <= probe_height] # take out any points where the measurement is larger than the probe height

df_new

,Date,Time,Latitude,Longitude,Elevation,Depth,geometry
0,02/11/2021,15:04:47,43.763237,-116.1047058,5335,0.00,None
2,02/11/2021,15:05:04,43.7632332,-116.1047058,5336,1.13,None


,Count,DistanceAvg,Distance1,Distance2,Distance3,Distance4,Distance5,latitude,longitude,altitude,sat_count,HDOP,dateTime,startGPSTime,endGPSTime,status,geometry
12,12,4985,4988,747,749,4985,4986,43.7616234,-116.1060257,5355,16,0.7,666372806,2021/02/11 15:33:25.100,2021/02/11 15:33:26.900,3,None
51,51,597,602,578,601,596,597,43.7574692,-116.102005,5630,13,1.0,666374575,2021/02/11 16:02:53.300,2021/02/11 16:02:55.200,3,None
61,61,4671,4671,4686,0,4678,0,43.7556953,-116.1005554,6112,10,2.0,666375264,2021/02/11 16:14:23.600,2021/02/11 16:14:24.600,3,None
35,35,0,0,0,0,0,0,43.7593918,-116.1047821,5443,17,0.8,666373622,2021/02/11 15:47:01.600,2021/02/11 15:47:02.100,3,None
103,103,0,0,0,0,0,0,43.7546501,-116.0998154,6153,17,0.8,666375485,2021/02/11 16:18:05.200,2021/02/11 16:18:05.600,3,None
6,6,677,700,683,677,674,677,43.7629089,-116.1048203,5345,18,0.7,666372452,2021/02/11 15:27:30.300,2021/02/11 15:27:32.200,3,None
31,31,0,0,0,0,0,0,43.7593651,-116.1049118,5436,15,0.8,666373600,2021/02/11 15:46:39.800,2021/02/11 15:46:40.400,3,None
62,62,0,0,0,0,0,0,43.7556877,-116.1005478,6113,11,1.3,666375266,2021/02/11 16:14:26.400,2021/02/11 16:14:26.800,3,None
74,74,4670,4670,4678,4678,0,0,43.7555351,-116.1004181,6121,9,1.7,666375309,2021/02/11 16:15:07.900,2021/02/11 16:15:09.300,3,None
53,53,4676,508,510,4677,4681,4676,43.757164,-116.1016693,5650,14,0.9,666374717,2021/02/11 16:05:15.600,2021/02/11 16:05:17.400,3,None
